<a href="https://colab.research.google.com/github/hatkiet/Project_4/blob/Fara_branch/Productionizing_using_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import io
import joblib
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from google.colab import files

In [2]:
# Upload a file from the local filesystem using Google Colab's file upload interface
uploaded_file = files.upload()

Saving ready_for_NN.csv to ready_for_NN.csv


In [3]:
# Extract the list of uploaded file names and display them
files_uploaded = list(uploaded_file.keys())
files_uploaded

['ready_for_NN.csv']

In [4]:
# Read the uploaded CSV file into a pandas DataFrame
df = pd.read_csv(io.BytesIO(uploaded_file[files_uploaded[0]]))
df

,State,Sex,Age,Race,BMI,GenHealth,PhysicalHealth,MentalHealth,PhysicalActivity,ChestScan,...,ECigaretteUsage,Drinking,HeartAttack,Angina,Stroke,Asthma,SkinCancer,Depressed,KidneyDisease,Diabetes
0,Alabama,Female,Age 80 or older,"White only, Non-Hispanic",26.57,Excellent,0.0,0.0,No,No,...,Never used e-cigarettes in my entire life,No,No,No,No,No,Yes,No,No,No
1,Alabama,Female,Age 55 to 59,"White only, Non-Hispanic",25.61,Very good,2.0,3.0,Yes,No,...,Never used e-cigarettes in my entire life,No,No,No,No,No,Yes,No,No,No
2,Alabama,Female,Age 40 to 44,"White only, Non-Hispanic",21.77,Fair,2.0,0.0,Yes,Yes,...,Never used e-cigarettes in my entire life,Yes,No,No,No,No,No,No,No,No
3,Alabama,Male,Age 80 or older,"White only, Non-Hispanic",26.08,Poor,1.0,0.0,No,No,...,Never used e-cigarettes in my entire life,No,Yes,No,Yes,No,No,No,No,Yes
4,Alabama,Female,Age 80 or older,"Black only, Non-Hispanic",22.96,Very good,0.0,0.0,Yes,No,...,Never used e-cigarettes in my entire life,Yes,No,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318929,Virgin Islands,Male,Age 65 to 69,"Multiracial, Non-Hispanic",29.86,Good,0.0,15.0,Yes,No,...,Never used e-cigarettes in my entire life,Yes,No,No,Yes,No,No,No,No,Yes
318930,Virgin Islands,Male,Age 30 to 34,"White only, Non-Hispanic",31.19,Good,0.0,0.0,Yes,No,...,Never used e-cigarettes in my entire life,Yes,No,No,No,No,No,No,No,No
318931,Virgin Islands,Female,Age 50 to 54,"Black only, Non-Hispanic",28.66,Excellent,2.0,2.0,Yes,No,...,Never used e-cigarettes in my entire life,No,No,No,No,No,No,No,No,No
318932,Virgin Islands,Male,Age 70 to 74,"Black only, Non-Hispanic",32.55,Very good,0.0,0.0,No,Yes,...,Never used e-cigarettes in my entire life,No,Yes,No,No,Yes,No,No,No,No


In [5]:
# Defining Columns to Keep for Feature Selection Models
important_features = df.columns.tolist()


In [6]:
# Creating DataFrame with Selected Columns and Target Column
df = df[important_features]
df["HeartAttack"] = df.HeartAttack


In [7]:
# Encoding Categorical Columns
encoder = LabelEncoder()
df["Angina"] = encoder.fit_transform(df.Angina)
df["HeartAttack"] = encoder.fit_transform(df["HeartAttack"])


In [8]:
# Saving Label Encoder to a File
joblib.dump(encoder, "label_encoder.pkl")

['label_encoder.pkl']

In [9]:
# Scaling Numeric Columns
scaler = MinMaxScaler()
col_to_scale = df.select_dtypes(include="float64")
scaled_columns = scaler.fit_transform(col_to_scale)
scaled_data = pd.DataFrame(scaled_columns, columns=col_to_scale.columns)

In [10]:
# Saving scaler to a File
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [11]:
# Creating Final Prepped Data by Concatenating Scaled and Encoded Columns
prepped_data = pd.concat([scaled_data, df[["Angina", "HeartAttack"]]], axis="columns")


In [12]:
# Split Data into Features and Target
X = prepped_data.drop(columns="HeartAttack")
y = prepped_data.HeartAttack

In [13]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, shuffle=True, random_state=42)


In [14]:
# Initialize a new sequential model using Keras
model = Sequential()

In [15]:
# Get the number of columns/features in the dataset
len(X.columns)

5

In [17]:
# Get the number of features/columns in the training dataset
X_train.shape[1]

5

In [19]:
# Create the neural network model
model = Sequential()
model.add(Dense(len(X.columns), input_shape=(X_train.shape[1],), activation="relu"))
model.add(Dense(42, activation="relu"))
model.add(Dense(21, activation="relu"))
model.add(Dense(1, activation="sigmoid"))


In [20]:
# Print a summary of the model architecture
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 30        
                                                                 
 dense_1 (Dense)             (None, 42)                252       
                                                                 
 dense_2 (Dense)             (None, 21)                903       
                                                                 
 dense_3 (Dense)             (None, 1)                 22        
                                                                 
Total params: 1207 (4.71 KB)
Trainable params: 1207 (4.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
# Compile the model with the specified settings
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [22]:
# Train the model
history = model.fit(X_train, y_train, batch_size=100, epochs=100, validation_split=0.2)

Epoch 1/100
1787/1787 [==============================] - 9s 2ms/step - loss: 0.1757 - accuracy: 0.9466 - val_loss: 0.1562 - val_accuracy: 0.9477
Epoch 2/100
1787/1787 [==============================] - 6s 3ms/step - loss: 0.1587 - accuracy: 0.9468 - val_loss: 0.1563 - val_accuracy: 0.9476
Epoch 3/100
1787/1787 [==============================] - 6s 3ms/step - loss: 0.1585 - accuracy: 0.9467 - val_loss: 0.1564 - val_accuracy: 0.9473
Epoch 4/100
1787/1787 [==============================] - 4s 2ms/step - loss: 0.1583 - accuracy: 0.9468 - val_loss: 0.1564 - val_accuracy: 0.9475
Epoch 5/100
1787/1787 [==============================] - 8s 4ms/step - loss: 0.1583 - accuracy: 0.9471 - val_loss: 0.1558 - val_accuracy: 0.9476
Epoch 6/100
1787/1787 [==============================] - 5s 3ms/step - loss: 0.1582 - accuracy: 0.9469 - val_loss: 0.1558 - val_accuracy: 0.9480
Epoch 7/100
1787/1787 [==============================] - 4s 2ms/step - loss: 0.1581 - accuracy: 0.9470 - val_loss: 0.1554 - val_ac

In [23]:
# Predict on the test set
y_hat = model.predict(X_test)

2991/2991 [==============================] - 4s 1ms/step


In [24]:
# Convert the predicted probabilities to binary class labels (0 or 1)
y_pres = (y_hat > 0.5).astype(int).flatten()

In [26]:
# Print the classification report to evaluate the performance of the model on the test set
print(classification_report(y_test,y_pres))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     90597
           1       0.54      0.14      0.22      5084

    accuracy                           0.95     95681
   macro avg       0.75      0.57      0.60     95681
weighted avg       0.93      0.95      0.93     95681



In [28]:
# Compute confusion matrix
confusion_matrix = tf.math.confusion_matrix(y_test, y_pres)

# Print the confusion matrix
print('Confusion Matrix:')
print(confusion_matrix.numpy())

Confusion Matrix:
[[89983   614]
 [ 4364   720]]


In [29]:
# Save the trained model to a file
model.save("heart_attack_prediction_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
